__Data notes__
- COVID cases
    - one file per measure (confirmed dealths, recovered)
    - one row per province/country
    - no duplicates from latlongs or multiple entries in same day
        - however, latlongs have dodgy numbers of digits so not very accurate: prob wont use these!
    - dates are columns, with new col added each day and zeros for missing data

Ideas
- add population data per country
- NOTE: lat/long were added to daily data on march 1st: jan and feb tables only have province/country!


# Imports and functions

In [1]:
import logging
import glob
import pandas as pd
import os
import geopandas as gpd
import json 
import datetime
import itertools
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.models import DateSlider, Select, HoverTool, DatetimeTickFormatter, NumeralTickFormatter
from bokeh.layouts import widgetbox, row, column
from bokeh.models.annotations import Title
from bokeh.models.widgets import Panel, Tabs
import colorcet 

In [2]:
os.chdir('/Users/jdorni/Documents/training/covid-19-bokeh/')

# Functions

In [3]:
def prepare_data(df, metric):
    """Clean and subset df by metric based on user input
    
    Args:
        df {pd.DataFrame} one row per country/date with country geometry and a column for each metric
        metric {str} one of ['Confirmed', 'Deaths', 'Recovered']
        
    Returns:
        df {pd.DataFrame}
    """
    
    # organise cols
    df.rename({'Country/Region': 'country', 'Province/State': 'province'}, axis=1, inplace=True)
    df.columns = [x.lower() for x in df.columns]
    df.fillna('No data', inplace=True)
    
    # convert from wide to long 
    # to get one row per province/country/date
    df = pd.melt(df, id_vars=['country', 'province', 'lat', 'long'],
                 var_name='day', value_name=metric)
    df.set_index(['country', 'province', 'lat', 'long', 'day'], inplace=True)
    
    return df.sort_values(by=['country', 'province'])

In [301]:
def dissolve_countries(gdf, iso_n3, main_country):
    """
    Dissolve polygons from more than one country into a single geometry
    For when one df has more granular country data than another, we dissolve
    to the largest scale
    """
    subset = gdf.loc[gdf['iso_n3'] == iso_n3].copy()
    subset['country'] = main_country   
    subset = subset.dissolve(by='iso_n3')
    subset.reset_index(inplace=True)
    tmp = gdf.loc[gdf['iso_n3'] != iso_n3].copy()
    return pd.concat([tmp, subset], sort=False)
    
def preprocess_geodata():
    
    ## Prepare country borders data
    
    # read in shapefile using Geopandas
    shapefile = 'data/country_boundaries/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3','ISO_N3','geometry']]

    # rename columns
    gdf.columns = ['country', 'country_code', 'iso_n3', 'geometry']
    
    gdf['iso_n3'] = gdf['iso_n3'].astype(int)

    # Replace -99 with proper ISO CODES
    gdf.loc[gdf['country'] == "Norway", ['iso_n3']] = 578
    gdf.loc[gdf['country'] == "Northern Cyprus", ['iso_n3']] = 196    # 196 is ISOCODE of Cyprus
    gdf.loc[gdf['country'] == "Somaliland", ['iso_n3']] = 706    # 706 is ISOCODE of Somalia 
    gdf.loc[gdf['country'] == "Kosovo", ['iso_n3']] = 688    # 688 is ISOCODE of Serbia 
    
    gdf = dissolve_countries(gdf, iso_n3=706, main_country='Somalia')
    gdf = dissolve_countries(gdf, iso_n3=688, main_country='Serbia')
    gdf = dissolve_countries(gdf, iso_n3=196, main_country='Cyprus')
    
    
    ## Add population (in thousands)
    # source: https://population.un.org/wpp/Download/Standard/CSV/
    
    population = pd.read_csv('data/populations/WPP2019_TotalPopulationBySex.csv')

    # select relevant data by time and scale
    population = population.loc[(population['Time'] == 2020) & 
                                (population['Variant'] == 'Medium')]   

    # add population to each country
    gdf = gdf.merge(population, right_on='LocID', left_on='iso_n3', how='left')

    # clean up
    gdf.drop(['LocID', 'iso_n3', 'Location', 'Time', 'Variant', 'VarID', 'MidPeriod', 'PopMale', 'PopFemale'], 
             axis=1, inplace=True, errors='ignore')

    return gdf


def preprocess_covid_data():
    """
    Prepare df for plotting
    
    - imports COVID-19 case and country boundary datasets
    - validates country names
    - joins datasets together
    - sets datatime data types
    - adds missing rows for countries with no cases reported on certain dates
    """

    confirmed = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    deaths = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
    recovered = pd.read_csv('data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
    
    confirmed_melted = prepare_data(df=confirmed, metric='confirmed')
    deaths_melted = prepare_data(df=deaths, metric='deaths')
    recovered_melted = prepare_data(df=recovered, metric='recovered')

    df = pd.concat([confirmed_melted, deaths_melted, recovered_melted], axis=1)
    df.reset_index(inplace=True)

    # sum provinces to get totals by whole countries  
    df = df.groupby(['country', 'day']).agg(
        confirmed=pd.NamedAgg(column='confirmed', aggfunc='sum'),
        deaths=pd.NamedAgg(column='deaths', aggfunc='sum'),
        recovered=pd.NamedAgg(column='recovered', aggfunc='sum'))

    df = df.reset_index()

    # convert date to a sortable string format
    df['day'] = pd.to_datetime(df['day']).dt.strftime("%Y-%m-%d")
    

    # remap country names (need to sum again after this as some countries are combined)
    
    country_map = {
        'Congo (Kinshasa)': 'Democratic Republic of the Congo',
        'Congo (Brazzaville)': 'Democratic Republic of the Congo',
        "Cote d'Ivoire": 'Ivory Coast',
        'Eswatini': 'eSwatini',
        'Gambia, The': 'Gambia', 
        'The Gambia': 'Gambia', 
        'Korea, South': 'South Korea', 
        'North Macedonia': 'Macedonia', 
        'Serbia': 'Republic of Serbia',
        'Taiwan*': 'Taiwan',
        'Tanzania': 'United Republic of Tanzania',
        'Timor-Leste': 'East Timor',
        'Bahamas, The': 'The Bahamas',
        'US': 'United States of America'
        #'Holy See': 'Italy'    # church jurisdiction and not country (vatican)
    }
    
#     # keep record of original country name
#     df['country_original'] = df['country']

    df.replace({"country": country_map}, inplace=True)
    
    return df


def preprocess_final_data(df, gdf):
    
    # left join so only countries we have geometries for are in the mapping dataset
    merged = gdf.merge(df, left_on = 'country', right_on = 'country', how='left')
    merged = merged.drop(['country_code'], axis=1)

    # re-convert day (including the NaN rows) to the correct string date format 
    merged['day'] = pd.to_datetime(merged['day']).dt.strftime("%Y-%m-%d")

    ## Add missing date rows for countries with no cases

    # create tmp table of all combinations of country X date
    country_date_cols = ['country', 'day']
    lists_of_uniques = [merged[col].unique() for col in country_date_cols]
    tmp = pd.DataFrame(list(itertools.product(*lists_of_uniques)), columns=country_date_cols)

    # add geometry from country borders
    tmp = tmp.merge(gdf[['country', 'geometry', 'PopTotal', 'PopDensity']], left_on = 'country', right_on = 'country', how='left')

    # outer join to add missing rows - will join on common columns
    merged = merged.merge(tmp, how='outer')

    # del rows with day='NaT' and NaNs
    merged = merged.loc[merged['day'] != 'NaT']
    merged.dropna(subset=['day'], inplace=True) #XXX CHECK THIS IS THE RIGHT THING TO DO!!

    # sort by date to plot time series
    merged.sort_values(by=['country', 'country_original', 'day'], inplace=True)
    
    # add cases per million people in population (population is in thousands)
    merged['confirmed_per_million'] = merged['confirmed'] / (merged['PopTotal']*1000)
    merged['deaths_per_million'] = merged['deaths'] / (merged['PopTotal']*1000)
    merged['recovered_per_million'] = merged['recovered'] / (merged['PopTotal']*1000)
    
    return merged

In [5]:
def source_by_date(data, selected_day):
    """Create geosource for date selected by user on slider. Returns pd.DataFrame."""
    selected_day = selected_day.strftime('%Y-%m-%d')
    new_data = data.loc[data['day'] == selected_day] 
    return new_data

In [6]:
# Bokeh functions

def slider_callback(attr, old, new):
    """Update numbers based on selected date"""
    day = date_slider.value

    # hack to convert bokeh epoch unix time to proper date: 
    # bokeh misses the decimal point so we must divide by 1000 - Antonio logged an issue
    day = datetime.datetime.fromtimestamp(day/1000)

    new_data = source_by_date(data, day)
    source.geojson = new_data.to_json() # overwrite the existing source's geojson

    
def menu_callback(attr, old, new):
    """Update color shading by selected metric"""
    
    if menu.value == 'Confirmed': 
        metric = 'confirmed'  
        color_mapper.palette = colorcet.b_linear_blue_5_95_c73[::-1]
        tooltips=[('Country', '@country'), ('Confirmed', '@confirmed{0,0}')]
        
    elif menu.value == 'Deaths':
        metric = 'deaths'
        color_mapper.palette = colorcet.b_linear_kry_5_98_c75[::-1]
        tooltips=[('Country', '@country'), ('Deaths', '@deaths{0,0}')]
        
    elif menu.value == 'Recovered': 
        metric = 'recovered'
        color_mapper.palette = colorcet.b_linear_green_5_95_c69[::-1]
        tooltips=[('Country', '@country'), ('Recovered', '@recovered{0,0}')]
        
    else:
        print('Unknown value')
        
    # update tooltips
    p1.hover.tooltips = tooltips
    
    # update colours
    vals = data[metric]    
    color_mapper.low = vals.min()
    color_mapper.high = vals.max()
    country_polygons.glyph.fill_color = {'field': metric, 'transform': color_mapper}
    color_bar.color_mapper = color_mapper

# Prepare data

In [302]:
gdf = preprocess_geodata()
df = preprocess_covid_data()
data = preprocess_final_data(df, gdf)

In [303]:
data.head()

,country,geometry,PopTotal,PopDensity,day,confirmed,deaths,recovered,country_original,confirmed_per_million,deaths_per_million,recovered_per_million
9119,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",38928.341,59.627,2020-01-22,0.0,0.0,0.0,Afghanistan,0.0,0.0,0.0
9120,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",38928.341,59.627,2020-01-23,0.0,0.0,0.0,Afghanistan,0.0,0.0,0.0
9121,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",38928.341,59.627,2020-01-24,0.0,0.0,0.0,Afghanistan,0.0,0.0,0.0
9122,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",38928.341,59.627,2020-01-25,0.0,0.0,0.0,Afghanistan,0.0,0.0,0.0
9123,Afghanistan,"POLYGON ((66.51861 37.36278, 67.07578 37.35614...",38928.341,59.627,2020-01-26,0.0,0.0,0.0,Afghanistan,0.0,0.0,0.0


# Explore World Population data

Total population by sex, annually from 1950 to 2100. Encoded in UTF-8.

Columns:
- LocID (numeric): numeric code for the location; for countries and areas, it follows the ISO 3166-1 numeric standard
- Location (string): name of the region, subregion, country or area
- VarID (numeric): numeric code for the variant 
- Variant (string): projection variant name (Medium is the most used); for more information see https://population.un.org/wpp/DefinitionOfProjectionVariants/
- Time (string): label identifying the single year (e.g. 1950) or the period of the data (e.g. 1950-1955)
- MidPeriod (numeric): numeric value identifying the mid period of the data, with the decimal representing the month (e.g. 1950.5 for July 1950)
- PopMale: Total male population (thousands)
- PopFemale: Total female population (thousands)
- PopTotal: Total population, both sexes (thousands)
- PopDensity: Population per square kilometre (thousands)




### Some countries have >1 row with the same LocID - THESE ARE ACTUALLY REGIONS SO IGNORE FOR NOW

In [19]:
population = pd.read_csv('data/populations/WPP2019_TotalPopulationBySex.csv')

In [20]:
# select relevant data by time and scale
population = population.loc[(population['Time'] == 2020) & (population['Variant'] == 'Medium')]

In [12]:
population.LocID.nunique(), population.Location.nunique()

(477, 474)

In [13]:
population.groupby('Location')['LocID'].nunique().reset_index().sort_values(by='LocID', ascending=False)

,Location,LocID
237,Latin America and the Caribbean,2
171,Europe,2
312,Northern America,2
311,Northern Africa and Western Asia,1
323,Organization of the Islamic Conference (OIC),1
...,...,...
154,Eastern Africa,1
153,East African Community (EAC),1
152,ESCWA: member countries,1
151,ESCWA: Mashreq countries,1


In [14]:
population.loc[population['Location'] == 'Latin America and the Caribbean']
population.loc[population['Location'] == 'Europe']
population.loc[population['Location'] == 'Northern America']

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
184625,918,Northern America,2,Medium,2020,2020.5,182580.522,186289.122,368869.644,19.777
184626,905,Northern America,2,Medium,2020,2020.5,182580.522,186289.122,368869.644,19.777


In [15]:
population.drop(['LocID', 'Time', 'Variant', 'VarID', 'MidPeriod'], axis=1, inplace=True, errors='ignore')

In [16]:
population.drop_duplicates(inplace=True)

In [17]:
population.loc[population['Location'] == 'Latin America and the Caribbean']
population.loc[population['Location'] == 'Europe']
population.loc[population['Location'] == 'Northern America']

,Location,PopMale,PopFemale,PopTotal,PopDensity
184625,Northern America,182580.522,186289.122,368869.644,19.777


### Correct -99 ISO country codes

In [ ]:
# Replace -99 with proper ISO CODES

# has one geometry
gdf.loc[gdf['country'] == "Norway", ['iso_n3']] = 578

# has multiple geometries - need to combine to one using geopandas 'dissolve' function
gdf.loc[gdf['country'] == "Northern Cyprus", ['iso_n3']] = 196    # 196 is ISOCODE of Cyprus
gdf.loc[gdf['country'] == "Somaliland", ['iso_n3']] = 706    # 706 is ISOCODE of Somalia (Somaliland is an area of Somalia)
gdf.loc[gdf['country'] == "Kosovo", ['iso_n3']] = 688    # 688 is ISOCODE of Serbia. Kosovo used to be part of Serbia and doesn't have its own ISOCODE yet. 
# Serbia's population includes Kosovo (according to wikipedia serbia's pop is 7M or 8.8M including kosovo. Here it's 8.7M).


In [138]:
# Combine geometries for countries that have one population but >1 geometry

def dissolve_countries(gdf, iso_n3, main_country):
    """Dissolve polygons from more than one country into a single geometry
    For when one df has more granular country data than another, we dissolve
    to the largest scale"""
    subset = gdf.loc[gdf['iso_n3'] == iso_n3].copy()
    subset['country'] = main_country   
    subset = subset.dissolve(by='iso_n3')
    subset.reset_index(inplace=True)
    
    # add row back in
    tmp = gdf.loc[gdf['iso_n3'] != iso_n3].copy()
    return pd.concat([tmp, subset], sort=False)
    
gdf = dissolve_countries(gdf, iso_n3=706, main_country='Somalia')
gdf = dissolve_countries(gdf, iso_n3=688, main_country='Serbia')
gdf = dissolve_countries(gdf, iso_n3=196, main_country='Cyprus')

In [139]:
len(gdf)  # lost 3 rows from dissolving polygons

174

In [140]:
gdf.loc[gdf['iso_n3'] == -99]

,country,country_code,iso_n3,geometry


# Plot

In [14]:
from bokeh.models.widgets import Panel, Tabs

tab1 = Panel(child=p1, title='Global Cases') # world map to show all countries, with date slider and metric toggle
tab2 = Panel(child=p2, title='Country Selector') # drop down to select country, with time series bar chart for each metric, and possibly table of data by province, and map of data by province

layout = Tabs(tabs=[tab1, tab2])
output_file('tabs.html')
show(layout)

NameError: name 'p1' is not defined

# PLOT 5: Covid data only (points, no map)

In [35]:
## points coloured by country (individual reports)

data = df.copy()
source = ColumnDataSource(data) # convert df to dict of colname: values (note all cols must be same length!)

# Make a CategoricalColorMapper 
n = data['country'].nunique()
print(n, len(data))
palette = inferno(n)
color_mapper = CategoricalColorMapper(factors=data['country'].unique(), palette=palette)

TOOLTIPS = [
    ('Country', '@country'),
    ('Province', '@province'),
    ('Confirmed', '@Confirmed'),
    ('Deaths', '@Deaths')
]

p = figure(plot_width=900, plot_height=500, title='test', tooltips=TOOLTIPS,
           x_axis_label='Longitude', y_axis_label='Latitude')
#p.hover.point_policy = "follow_mouse"

p.circle('Longitude','Latitude', source=source, 
         color=dict(field='country', transform=color_mapper), size=5)

output_notebook()
show(p)

182 14168


NameError: name 'inferno' is not defined

In [57]:
# sum provinces to get totals by whole countries  

df_agg = df.groupby('country').agg(
    confirmed=pd.NamedAgg(column='Confirmed', aggfunc='sum'),
    deaths=pd.NamedAgg(column='Deaths', aggfunc='sum'),
    recovered=pd.NamedAgg(column='Recovered', aggfunc='sum'),
    Latitude=pd.NamedAgg(column='Latitude', aggfunc='mean'),
    Longitude=pd.NamedAgg(column='Longitude', aggfunc='mean'),
    n_rows=pd.NamedAgg(column='Last Update', aggfunc='count'))

df_agg = df_agg.reset_index()
df_agg.head(6)

,Country/Region,confirmed,deaths,recovered,Latitude,Longitude,n_rows
0,Afghanistan,22,0,1,33.9391,67.7100,1
1,Albania,59,2,0,41.1533,20.1683,1
2,Algeria,74,7,12,28.0339,1.6596,1
3,Andorra,39,0,1,42.5063,1.5218,1
4,Antigua and Barbuda,1,0,0,17.0608,-61.7964,1
5,Argentina,79,2,3,-38.4161,-63.6167,1


In [72]:
# points coloured by number of cases (in prog)


data = df_agg.copy()
source = ColumnDataSource(data)

# Create a ColumnDataSource from df: source
source = ColumnDataSource(data)  
 
# Make a CategoricalColorMapper 
n = data['Country/Region'].nunique()
print(n, len(data))
palette = inferno(n)
color_mapper = CategoricalColorMapper(factors=data['Country/Region'].unique(), palette=palette)

p = figure(plot_width=900, plot_height=500, title='test', x_axis_label='Longitude', y_axis_label='Latitude')

p.circle('Longitude','Latitude', source=source, 
         color=dict(field='Country/Region', transform=color_mapper), size=5)

output_notebook()
show(p)

163 163


Loading BokehJS ...

# Time series

In [36]:
# source for plotting bars

# subset by country
country_key = 'United Kingdom'
merged_subset = merged.loc[merged['country'] == country_key].copy()

# convert day strings to datetimes for plotting as time series
merged_subset['day'] = pd.to_datetime(merged_subset['day'])
merged_subset.drop('geometry', axis=1, inplace=True)
merged_subset.head()

,country,day,confirmed,deaths,recovered,country_original
10025,United Kingdom,2020-01-22,0.0,0.0,0.0,United Kingdom
10026,United Kingdom,2020-01-23,0.0,0.0,0.0,United Kingdom
10027,United Kingdom,2020-01-24,0.0,0.0,0.0,United Kingdom
10028,United Kingdom,2020-01-25,0.0,0.0,0.0,United Kingdom
10029,United Kingdom,2020-01-26,0.0,0.0,0.0,United Kingdom


In [38]:
#source = ColumnDataSource(merged_subset) # coldatasources must exclude geometry column
df = merged_subset

metric = 'confirmed'

p = figure(title='Country', plot_height=400 , plot_width=850, 
           toolbar_location='right', 
           tools='wheel_zoom, pan, reset', 
           x_axis_label='Date', y_axis_label='Count',
        x_axis_type='datetime')

# p.xgrid.grid_line_color = None
# p.ygrid.grid_line_color = None

p.line(df['day'], df[metric])

# format the date on x axis
p.xaxis.formatter = DatetimeTickFormatter(days=["%d %b"])
    
# add thousand-separator to y axis
p.yaxis.formatter=NumeralTickFormatter() 


# Create a HoverTool: hover
hover = HoverTool(tooltips=TOOLTIPS)

# Add the hover tool to the figure p
p.add_tools(hover)


output_notebook()
show(p)

Loading BokehJS ...

# Map by province

In [51]:
# # source for mapping by province #TODO - need to unaggregate by province

#merged_subset = merged.loc[merged['country'] == 'United Kingdom']

# source = merged_subset.copy()
# source = GeoJSONDataSource(geojson=json.dumps(json.loads(source.to_json()))) 